In [1]:
import sys
print(sys.executable)

/home/tibia/miniconda3/envs/libmtl/bin/python


# Préparation du dataloader


In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd

SEG_LABEL_COLS = ['any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']
SEG_DIR = '/home/tibia/Projet_Hemorragie/Seg_hemorragie/split_MONAI'
CLASSIFICATION_DATA_DIR = '/home/tibia/Projet_Hemorragie/MBH_label_case'
SAVE_DIR = "/home/tibia/Projet_Hemorragie/MBH_multitask_libMTL/saved_models"
os.makedirs(SAVE_DIR, exist_ok=True)
# ======================
# DATA PREPARATION
# ======================
def get_segmentation_data(split="train"):
    img_dir = Path(SEG_DIR) / split / "img"
    seg_dir = Path(SEG_DIR) / split / "seg"
    
    images = sorted(img_dir.glob("*.nii.gz"))
    labels = sorted(seg_dir.glob("*.nii.gz"))
    
    assert len(images) == len(labels), "Mismatch between image and label counts"

    data = []
    for img, lbl in zip(images, labels):
        data.append({
            "image": str(img),
            "label": str(lbl),
        })
        
    return data


def get_classification_data(split="train"):
    csv_path = Path(CLASSIFICATION_DATA_DIR) / "splits" / f"{split}_split.csv"
    df = pd.read_csv(csv_path)
    nii_dir = Path(CLASSIFICATION_DATA_DIR)
    label_cols = ['any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']
    
    data = []
    for _, row in df.iterrows():
        image_path = str(nii_dir / f"{row['patientID_studyID']}.nii.gz")
        label = np.array([row[col] for col in label_cols], dtype=np.float32)
        
        data.append({
            "image": image_path,
            "label": label
        })
    return data

In [10]:

from monai.data import DataLoader, PersistentDataset, CacheDataset
import torch
import os
import monai.transforms as T
import random

batch_size = 2
seg_train_data=get_segmentation_data("train")
cls_train_data=get_classification_data("train")

#val à faire plus tard

# Transforms à appliquer après
#train_transforms, val_transforms = T_mtsk.TaskBasedTransform_V2(keys=["image", "label"]), T_mtsk.TaskBasedValTransform_V2(keys=["image", "label"])
    
seg_transforms = T.Compose([
            T.LoadImaged(keys=["image", "label"], image_only=True),
            T.EnsureChannelFirstd(keys=["image", "label"]) ])

cls_transforms = T.Compose([  
            T.LoadImaged(keys=["image"], image_only=True),
            T.EnsureChannelFirstd(keys=["image"]) ])  # Pas de label à charger pour la classification ici

    # Datasets
seg_train_dataset = PersistentDataset(
        seg_train_data, 
        transform=seg_transforms,
        cache_dir=os.path.join(SAVE_DIR, "cache_train")
    )

cls_train_dataset = PersistentDataset(
        cls_train_data,
        transform=cls_transforms,
        cache_dir=os.path.join(SAVE_DIR, "cache_train"))
    
#Val dataset à faire plus tard

# DataLoaders
seg_train_loader = DataLoader(
        seg_train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=8,
        persistent_workers=True,
)

cls_train_loader = DataLoader(
        cls_train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=8,
        persistent_workers=True,
)
  
train_dataloaders = {'seg': seg_train_loader,
                     'cls': cls_train_loader
                     }

print(f"Nombre de batches dans le DataLoader de segmentation (train) : {len(train_dataloaders['seg'])}")
print(f"Nombre de batches dans le DataLoader de classification (train) : {len(train_dataloaders['cls'])}")

Nombre de batches dans le DataLoader de segmentation (train) : 77
Nombre de batches dans le DataLoader de classification (train) : 637


In [4]:
import sys
import os

# Définir le chemin vers le répertoire 'LibMTL' de la bibliothèque
# On utilise os.path.abspath et os.path.join pour assurer la bonne formation du chemin
# Dans votre cas, le chemin est : /home/tibia/Projet_Hemorragie/LibMTL/LibMTL/LibMTL
libmtl_path = '/home/tibia/Projet_Hemorragie/LibMTL/LibMTL'

# Vérifiez si le chemin est déjà dans sys.path pour éviter la duplication
if libmtl_path not in sys.path:
    # Ajouter le chemin au sys.path
    sys.path.insert(0, libmtl_path)
    print(f"Le chemin '{libmtl_path}' a été ajouté à sys.path.")
else:
    print(f"Le chemin '{libmtl_path}' est déjà dans sys.path.")

Le chemin '/home/tibia/Projet_Hemorragie/LibMTL/LibMTL' a été ajouté à sys.path.


## Préparation dictionnaire de tâche 

In [ ]:
# Loss
from LibMTL.metrics import AbsMetric
from monai.metrics import DiceMetric,DiceHelper
from monai.utils import MetricReduction, deprecated_arg
        
class DiceMetricAdapter(AbsMetric):
    """
    Cet adaptateur implémente AbsMetric pour calculer le Dice Score correctement.
    
    - `update_fun` utilise DiceHelper pour obtenir les scores bruts (B, C) 
      et les stocke dans `self.record`.
    - `score_fun` agrège tous les scores de `self.record` et calcule 
      la moyenne finale (le "score des totaux" émulé).
    """
    def __init__(self, num_classes, include_background=False):
        # Initialise self.record et self.bs
        super().__init__()
        
        self.num_classes = num_classes
        self.include_background = include_background
        
        # On utilise DiceHelper comme "calculateur" ponctuel.
        # On lui demande de NE PAS faire de réduction (reduction="none")
        # car on veut stocker les scores bruts (Batch, Classes).
        self.dice_helper = DiceHelper(
            include_background=include_background,
            num_classes=num_classes,
            reduction=MetricReduction.NONE,
            ignore_empty=True,  # Important : ignore les cas où le GT est vide
            apply_argmax=False  # On le fera nous-mêmes dans update_fun
        )

    def update_fun(self, pred, gt):
        """
        Appelé à chaque batch. Calcule les scores (B, C) et les stocke.
        
        Args:
            pred (torch.Tensor): Prédictions (logits) de forme (B, C, H, W, D)
            gt (torch.Tensor): Vérité terrain (labels) de forme (B, 1, H, W, D)
        """
        # 1. Convertir les logits en labels
        # DiceHelper attend des labels, pas des logits
        pred_labels = torch.argmax(pred, dim=1, keepdim=True)
        
        # 2. Calculer les scores Dice pour ce batch
        # Le résultat est un tenseur de (B, num_classes_calculées)
        # ex: (B, 5) si num_classes=6 et include_background=False
        batch_dice_scores,_ = self.dice_helper(pred_labels, gt)
        
        # 3. Stocker ce tenseur dans notre "record"
        self.record.append(batch_dice_scores)
        
        # 4. Stocker la taille du batch (comme le fait AbsMetric)
        self.bs.append(pred.shape[0])

    def score_fun(self):
        """
        Appelé à la fin de l'époque. Agrège les scores et calcule la moyenne. Peut etre à modifier pour le loggage de chaque dice
        """
        if not self.record:
            # Retourne un score pour chaque classe, mis à 0
            num_expected_classes = self.num_classes - (1 if not self.include_background else 0)
            return torch.zeros(num_expected_classes)
            
        # 1. Rassembler tous les tenseurs de (B, C) en un seul
        # grand tenseur de (Total_B, C)
        all_scores = torch.cat(self.record, dim=0)
        
        # 2. Calculer la moyenne sur la dimension des batches (dim=0)
        # On utilise nanmean pour ignorer les NaN (cas des GT vides)
        # C'est la façon correcte d'agréger le Dice.
        mean_scores_per_class = torch.nanmean(all_scores, dim=0)
        
        # `score_fun` est censé retourner une "liste", mais un tenseur
        # est plus utile. On retourne la moyenne par classe.
        return mean_scores_per_class
    
    # La méthode reinit() est héritée de AbsMetric et fonctionne parfaitement
    # car elle vide self.record et self.bs.


In [ ]:
# Model
from monai.networks import nets as monai_nets

model = monai_nets.UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=6,
    channels=(32, 64, 128, 256, 320, 320),
    strides=(2, 2, 2, 2, 2),
    num_res_units=2,
)

In [12]:
import torch

B = 2    # Batch Size (nombre d'échantillons)
C = 6    # Nombre de Classes (+ background)
H = 32   # Hauteur (Height)
W = 32   # Largeur (Width)
D = 16   # Profondeur (Depth)


pred = torch.rand(B, C, H, W, D) 
pred_labels = torch.argmax(pred, dim=1, keepdim=True)
pred_lables_2= torch.nn.Softmax(dim=1)(pred)

print("Predicted Labels Shape:", pred_labels.shape)  # Devrait afficher (2, 1, 32, 32, 16)
print("Predicted Labels after Softmax Shape:", pred_lables_2.shape)  # Devrait afficher (2, 6, 32, 32, 16)
# --- Tenseur de Vérité Terrain (Labels) ---
# Forme désirée : (B, 1, H, W, D) -> (2, 1, 32, 32, 16)
# Utilisation de torch.randint pour simuler des labels (entiers de 0 à C-1)
# Les labels doivent être des entiers et non des flottants.
gt = torch.randint(low=0, high=C, size=(B, 1, H, W, D))



Predicted Labels Shape: torch.Size([2, 1, 32, 32, 16])
Predicted Labels after Softmax Shape: torch.Size([2, 6, 32, 32, 16])


In [ ]:
# Losses
# Ponderer ensuite pa classe avec WeightSampler

from LibMTL.loss import AbsLoss
import torch
from monai.losses import DiceCELoss

class ClassificationLossWrapper(AbsLoss):
    def __init__(self):
        super().__init__()
        self.loss_fn = torch.nn.BCEWithLogitsLoss()

    def compute_loss(self, pred, gt):
        return self.loss_fn(pred, gt.float())
    
class SegmentationLossWrapper(AbsLoss):
    def __init__(self):
        super().__init__()
        self.loss_fn = DiceCELoss(
            include_background=False,
            to_onehot_y=True,
            softmax=True
        )

    def compute_loss(self, pred, gt):
        return self.loss_fn(pred, gt)

In [4]:
from LibMTL.metrics import AbsMetric
import torch

from torchmetrics.classification import MultilabelAUROC

class MultiLabelAUCMetric(AbsMetric):
    def __init__(self, num_labels=6):
        super().__init__()
        self.metric = MultilabelAUROC(num_labels=num_labels, average=None)   # par classe
        self.metric_mean = MultilabelAUROC(num_labels=num_labels, average="macro")  # moyenne
        self.num_labels = num_labels

    def update_fun(self, pred, gt):
        # pred = logits -> transform needed
        pred = torch.sigmoid(pred)
        self.metric.update(pred.detach().cpu(), gt.detach().cpu())
        self.metric_mean.update(pred.detach().cpu(), gt.detach().cpu())

    def score_fun(self):
        per_class = self.metric.compute().tolist()
        mean_auc = self.metric_mean.compute().item()
        return per_class + [mean_auc]

    def reinit(self):
        super().reinit()
        self.metric.reset()
        self.metric_mean.reset()

/home/tibia/miniconda3/envs/libmtl/lib/python3.10/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /store/home/tibia/miniconda3/envs/libmtl/lib/python3.10/site-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/home/tibia/miniconda3/envs/libmtl/lib/python3.10/site-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /store/home/tibia/miniconda3/envs/libmtl/lib/python3.10/site-packages/torch_sparse/_version_cuda.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [ ]:
# dictionnaire de tâches
task_dict = {
    'classification': {
        'loss_fn': ClassificationLossWrapper(),
        'metrics_fn': MultiLabelAUCMetric(num_labels=6),
        'metrics': ['AUROC'],
        'weight': [1.0]
    },
    'segmentation': {
        'loss_fn': SegmentationLossWrapper(),
        'metrics_fn': DiceMetricAdapter(num_classes=6, include_background=False),
        'metrics': ['Dice_per_class'],
        'weight': [1.0]
    }
}

In [ ]:
import torch
import torch.nn as nn
from monai.losses import DiceCELoss
from monai.metrics import DiceHelper
from torchmetrics.classification import MultilabelAUROC

# Background + 5 classes
NUM_LABELS_CLASSIF = 6 
NUM_CLASSES_SEG = 6 

#Fonctions de pertes
loss_classif = torch.nn.BCEWithLogitsLoss()
loss_seg = DiceCELoss(include_background=False, to_onehot_y=True, softmax=True)

#Métriques
cls_auc = MultilabelAUROC(num_labels=NUM_LABELS_CLASSIF, average=None)
cls_mean_auc = MultilabelAUROC(num_labels=NUM_LABELS_CLASSIF)

dice_metric = DiceHelper(
            include_background=False,
            softmax=True,
            num_classes=6,
            reduction='none'
        )


# --- 4. Construire le dictionnaire final ---

task_dict = {
    'classification': {
        'loss_fn': loss_classif,
        'metrics_fn': cls_auc,
        'metrics': ['AUROC'],
        'weight': [1.0]             
    },
    'segmentation': {
        'loss_fn': loss_seg,
        'metrics_fn': dice_metric,
        'metrics': ['Dice_per_class'],   
        'weight': [1.0]
    }
}

print("task_dict défini avec succès :")
print(task_dict)

task_dict défini avec succès :
{'classification': {'loss_fn': BCEWithLogitsLoss(), 'metrics_fn': MultilabelAUROC(), 'metrics': ['AUROC'], 'weight': [1.0]}, 'segmentation': {'loss_fn': DiceCELoss(
  (dice): DiceLoss()
  (cross_entropy): CrossEntropyLoss()
  (binary_cross_entropy): BCEWithLogitsLoss()
), 'metrics_fn': <monai.metrics.meandice.DiceHelper object at 0x7f3e711b1190>, 'metrics': ['Dice_per_class'], 'weight': [1.0]}}


In [ ]:
from typing import Sequence
import torch
import torch.nn as nn
from monai.networks.nets.basic_unet import TwoConv, Down, UpCat


class HemorrhageEncoder(nn.Module):
    """
    Cette classe contient la partie descendante (encodeur) du U-Net.
    Elle est partagée par les deux tâches.
    Son forward pass retourne une liste de toutes les feature maps
    nécessaires pour les skip connections du décodeur de segmentation.
    """
    def __init__(
        self,
        spatial_dims: int = 3,
        in_channels: int = 1,
        features: Sequence[int] = (32, 32, 64, 128, 256, 32),
        act: str | tuple = ("LeakyReLU", {"negative_slope": 0.1, "inplace": True}),
        norm: str | tuple = ("instance", {"affine": True}),
        bias: bool = True,
        dropout: float | tuple = 0.0,
    ):
        super().__init__()
        
        # Assure que 'features' a la bonne longueur
        self.fea = nn.Parameter(torch.tensor(features), requires_grad=False)
        
        self.conv_0 = TwoConv(spatial_dims, in_channels, self.fea[0], act, norm, bias, dropout)
        self.down_1 = Down(spatial_dims, self.fea[0], self.fea[1], act, norm, bias, dropout)
        self.down_2 = Down(spatial_dims, self.fea[1], self.fea[2], act, norm, bias, dropout)
        self.down_3 = Down(spatial_dims, self.fea[2], self.fea[3], act, norm, bias, dropout)
        self.down_4 = Down(spatial_dims, self.fea[3], self.fea[4], act, norm, bias, dropout)

    def forward(self, x: torch.Tensor) -> list[torch.Tensor]:
        """
        Le forward pass de l'encodeur.
        Retourne une liste contenant le bottleneck (x4) et toutes les
        sorties intermédiaires pour les skip connections.
        """
        x0 = self.conv_0(x)
        x1 = self.down_1(x0)
        x2 = self.down_2(x1)
        x3 = self.down_3(x2)
        x4 = self.down_4(x3)  # C'est le bottleneck (la représentation partagée)
        
        return [x4, x3, x2, x1, x0]

# ========================================================================
# 2. LES DÉCODEURS (Les têtes spécifiques à chaque tâche)
# ========================================================================

class SegmentationDecoder(nn.Module):
    """
    Le décodeur pour la tâche de segmentation.
    Il prend la liste de features de l'encodeur et reconstruit le masque.
    """
    def __init__(
        self,
        spatial_dims: int = 3,
        out_channels: int = 6,
        features: Sequence[int] = (32, 32, 64, 128, 256, 32),
        act: str | tuple = ("LeakyReLU", {"negative_slope": 0.1, "inplace": True}),
        norm: str | tuple = ("instance", {"affine": True}),
        bias: bool = True,
        dropout: float | tuple = 0.0,
        upsample: str = "deconv",
    ):
        super().__init__()
        
        fea = nn.Parameter(torch.tensor(features), requires_grad=False)
        
        self.upcat_4 = UpCat(spatial_dims, fea[4], fea[3], fea[3], act, norm, bias, dropout, upsample)
        self.upcat_3 = UpCat(spatial_dims, fea[3], fea[2], fea[2], act, norm, bias, dropout, upsample)
        self.upcat_2 = UpCat(spatial_dims, fea[2], fea[1], fea[1], act, norm, bias, dropout, upsample)
        self.upcat_1 = UpCat(spatial_dims, fea[1], fea[0], fea[5], act, norm, bias, dropout, upsample, halves=False)
        self.final_conv = nn.Conv3d(fea[5], out_channels, kernel_size=1)

    def forward(self, enc_out: list[torch.Tensor]) -> torch.Tensor:
        # On récupère les tenseurs de la liste fournie par l'encodeur
        x4, x3, x2, x1, x0 = enc_out
        
        u4 = self.upcat_4(x4, x3)
        u3 = self.upcat_3(u4, x2)
        u2 = self.upcat_2(u3, x1)
        u1 = self.upcat_1(u2, x0)
        
        return self.final_conv(u1)

class ClassificationDecoder(nn.Module):
    """
    Le décodeur pour la tâche de classification.
    Il prend la liste de features de l'encodeur mais n'utilise que le
    bottleneck (x4) pour prédire les classes.
    """
    def __init__(
        self,
        in_features: int,  # Doit correspondre à features[4] de l'encodeur
        num_cls_classes: int = 6,
    ):
        super().__init__()
        
        # Tête de classification, exactement comme avant
        self.cls_head = nn.Sequential(
            nn.AdaptiveAvgPool3d((4, 4, 4)),
            nn.Flatten(),
            nn.Linear(in_features * 4 * 4 * 4, 512),
            nn.LayerNorm(512),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, 256),
            nn.LayerNorm(256),
            nn.LeakyReLU(inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, num_cls_classes)
        )

    def forward(self, enc_out: list[torch.Tensor]) -> torch.Tensor:
        # On ne prend que le bottleneck (le premier élément de la liste)
        x4 = enc_out[0]
        
        # Toute la logique d'agrégation de patches a disparu !
        # On passe directement les features à la tête de classification.
        return self.cls_head(x4)

# ========================================================================
# 3. ASSEMBLAGE FINAL POUR LibMTL
# ========================================================================

# Définis tes paramètres
task_name = ["segmentation", "classification"]
features = (32, 32, 64, 128, 256, 32)

# Crée une instance de l'encodeur partagé
encoder = HemorrhageEncoder(features=features)

# Crée un dictionnaire de décodeurs
decoders = nn.ModuleDict({
    'segmentation': SegmentationDecoder(
        out_channels=6, # 6 classes de segmentation
        features=features
    ),
    'classification': ClassificationDecoder(
        in_features=features[4], # La taille du bottleneck (256)
        num_cls_classes=6 # 6 classes de classification
    )
})



In [ ]:
from LibMTL.architecture import SharedBottom
from LibMTL.weighting import Equal
import wandb

config_l = dict(
    sharing_type="hard",   # "soft" ou "fine_tune"
    model="BasicUNetWithClassification",
    loss_weighting="none",
    dataset_size="balanced",  # "full" ou "balanced" ou "optimized"
    batch_size=2,
    learning_rate=1e-3,
    optimizer="sgd",
    seed=42
)
torch.cuda.set_device(0)
# Génération automatique de tags à partir de config
tags = [f"{k}:{v}" for k, v in config_l.items() if k in ["sharing_type", "optimizer", "model", "loss_weighting"]]

# ⬇️ CHANGEMENT : Initialisation manuelle de wandb
# Au lieu de : wandb_logger = WandbLogger(...)
wandb.init(
    project="hemorrhage_multitask_test",
    group="noponderation",
    tags=tags,
    config=config_l,
    name="multitask_unet3d_libMTL"
)

# Paramètres optim & scheduler
optim_param = {
    'method': 'SGD', 
    'lr': 1e-3, 
    'weight_decay': 3e-5,  # 0.00003 est égal à 3e-5
    'momentum': 0.99, 
    'nesterov': True
}

scheduler_param = {
    'method': 'LinearScheduleWithWarmup',  # Correspond à get_linear_schedule_with_warmup
    'num_warmup_steps': 0, 
    'num_training_steps': "VOTRE_TOTAL_STEPS" # Ceci correspond à self.num_steps
}

# 3️ Méthodes multitâches
from LibMTL.architecture import HPS
from LibMTL.weighting import GradNorm

# 4️ Instanciation du Trainer
from LibMTL import Trainer

hemorrhage_trainer = Trainer(
    task_dict=task_dict,
    weighting=None,
    architecture=HPS,
    encoder_class=HemorrhageEncoder,
    decoders=decoders,
    rep_grad=False,
    multi_input=True,
    optim_param=optim_param,
    scheduler_param=scheduler_param,
    device='cuda'
)

# 5️⃣ Entraînement
hemorrhage_trainer.train(train_dataloaders, num_epochs=100)